# A Pytorch Implementation of SinGAN

In [ ]:
import os
from typing import Optional, Tuple, List
import torch
from torch import nn
import torch.nn.functional as F
import math
from tqdm import tqdm
import matplotlib.pyplot as plt

## Basic Building Block

In [2]:
class ConvBlock(nn.Module):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: int,
                 stride: int,
                 padding: int) -> None:
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.norm = nn.BatchNorm2d(out_channels)
        self.relu = nn.LeakyReLU(0.2, True)
    
    def forward(self,
                x: torch.Tensor) -> torch.Tensor:
        return self.relu(self.norm(self.conv(x)))

## Discriminator

In [11]:
class Discriminator(nn.Module):
    def __init__(self,
                 in_channels: int = 3,
                 out_channels: int = 32,
                 kernel_size: int = 3,
                 padding: int = 1,
                 num_layers: int = 3) -> None:
        super(Discriminator, self).__init__()
        self.head = ConvBlock(in_channels, out_channels, kernel_size, 1, padding)
        self.body = nn.Sequential(
            *[ConvBlock(out_channels, out_channels, kernel_size, 1, padding)
            for _ in range(num_layers)]
        )
        self.tail = ConvBlock(out_channels, 1, kernel_size, 1, padding)
        # self.pad = nn.ZeroPad2d(5)
    
    def forward(self,
                x: torch.Tensor) -> torch.Tensor:
        # x = self.pad(x)
        x = self.head(x)
        x = self.body(x)
        return self.tail(x)

## Generator

In [4]:
class Generator(nn.Module):
    def __init__(self,
                 in_channels: int = 3,
                 out_channels: int = 32,
                 kernel_size: int = 3,
                 padding: int = 1,
                 num_layers: int = 3) -> None:
        super(Generator, self).__init__()
        self.head = ConvBlock(in_channels, out_channels, kernel_size, 1, padding)
        self.body = nn.Sequential(
            *[ConvBlock(out_channels, out_channels, kernel_size, 1, padding)
            for _ in range(num_layers)]
        )
        self.tail = nn.Sequential(
            ConvBlock(out_channels, in_channels, kernel_size, 1, padding),
            nn.Tanh()
        )
        # self.pad = nn.ZeroPad2d(5)
    
    def forward(self,
                noise: torch.Tensor,
                x: torch.Tensor) -> torch.Tensor:
        # x = self.pad(x)
        return x + self.tail(self.body(self.head(x + noise)))

## Read & Write Image

In [42]:
def read_image(path: str) -> torch.Tensor:
    image = plt.imread(path)
    image = torch.from_numpy(image.copy()).float() / 255.0
    image = image.permute(2, 0, 1)
    return (image - 0.5) * 2

def write_image(path: str,
                image: torch.Tensor) -> None:
    image = (image + 1) / 2
    image = image.clamp(0, 1)
    plt.imsave(path, image.permute(1, 2, 0).cpu().numpy())

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = '../data/angkorwat.jpg'
image = read_image(path).to(device)
max_size, min_size = 250, 25

## Create Image Pyramid

In [ ]:
scale_factor = 3 / 4
number_of_scales = int(math.log(min_size / max_size, scale_factor)) # 8
number_of_scales

8

In [8]:
def adjust_scales2image(image: torch.Tensor) -> torch.Tensor:
    # resize the long side of the image to max_size
    scale = min(max_size / max([image.shape[2], image.shape[3]]), 1)
    real = F.interpolate(image, scale_factor=scale, mode='bilinear', align_corners=True)
    return real
real = adjust_scales2image(image.unsqueeze(0))
print(real.shape)

torch.Size([1, 3, 166, 250])


In [9]:
def create_reals_pyramid(real: torch.Tensor,
                         scale_factor: float,
                         stop_scale: int = 7):
    reals = []
    for i in range(stop_scale):
        scale = math.pow(scale_factor, ((stop_scale - 1) / math.log(stop_scale)) * math.log(stop_scale - i) + 1)
        curr_real = F.interpolate(real, scale_factor=scale, mode='bilinear', align_corners=True)
        reals.append(curr_real)
    reals.append(real)
    return reals

reals = create_reals_pyramid(real, scale_factor, number_of_scales)
for r in reals:
    print(r.shape)

torch.Size([1, 3, 16, 25])
torch.Size([1, 3, 18, 28])
torch.Size([1, 3, 21, 33])
torch.Size([1, 3, 26, 39])
torch.Size([1, 3, 32, 48])
torch.Size([1, 3, 42, 64])
torch.Size([1, 3, 63, 95])
torch.Size([1, 3, 124, 187])
torch.Size([1, 3, 166, 250])


## Train

In [43]:
n_channels = image.size(0)
n_features = 32
generators = [Generator(in_channels=n_channels, out_channels=n_features * pow(2, i // 4)).to(device) for i in range(number_of_scales + 1)]
discriminators = [Discriminator(in_channels=n_channels, out_channels=n_features * pow(2, i // 4)).to(device) for i in range(number_of_scales + 1)]

lr = 5e-4
beta1 = 0.5
gamma = 0
max_epochs = 2000
d_iter = g_iter = 3
alpha = 10.0
lamb = 0.1

reals_shapes = [r.shape[2:] for r in reals]
fixed_noise = []
noise_amp = []

In [16]:
output_dir = 'output'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
for i in range(number_of_scales + 1):
    p = f'{output_dir}/stage_{i:02d}'
    if not os.path.exists(p):
        os.mkdir(p)

In [ ]:
def generate(generators: List[Generator],
             noise: torch.Tensor,
             reals_shapes: List[torch.Size],
             noise_amp: List[float]) -> torch.Tensor:
    x = generators[0](noise[0], 0)
    for i in range(1, len(noise)):
        x = F.interpolate(x, size=reals_shapes[i], mode='bilinear', align_corners=True)
        x = generators[i](noise[i] * noise_amp[i], x)
    return x

In [15]:
def calc_gradient_penalty(discriminator: Discriminator,
                          real_data: torch.Tensor,
                          fake_data: torch.Tensor,
                          lamb: float = 0.1,
                          device: str = 'cpu') -> torch.Tensor:
    alpha = torch.rand(1).item()
    interpolates = alpha * real_data + (1 - alpha) * fake_data
    interpolates.requires_grad_(True)
    disc_interpolates = discriminator(interpolates)
    gradients = torch.autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                                    grad_outputs=torch.ones_like(disc_interpolates, device=device),
                                    create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lamb
    return gradient_penalty

In [44]:
for stage, (generator, discriminator, real) in enumerate(zip(generators, discriminators, reals)):
    print(f'Stage: {stage}')
    if stage % 4:
        generator.load_state_dict(torch.load(f'{output_dir}/stage_{stage - 1:02d}/generator.pth'))
        discriminator.load_state_dict(torch.load(f'{output_dir}/stage_{stage - 1:02d}/discriminator.pth'))
    
    z_opt = torch.randn(1, n_channels, *reals_shapes[stage], device=device) if stage == 0 else torch.zeros_like(real, device=device)
    fixed_noise.append(z_opt)
    
    optim_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
    optim_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
    scheduler_g = torch.optim.lr_scheduler.MultiStepLR(optim_g, milestones=[1600], gamma=gamma)
    scheduler_d = torch.optim.lr_scheduler.MultiStepLR(optim_d, milestones=[1600], gamma=gamma)
    
    # calculate noise_amp
    if stage == 0:
        noise_amp.append(1)
    else:
        noise_amp.append(0)
        with torch.no_grad():
            z_resconstruction = generate(generators, fixed_noise, reals_shapes, noise_amp)
        rec_loss = F.mse_loss(z_resconstruction, real)
        noise_amp[-1] = 0.1 * torch.sqrt(rec_loss)
    
    for epoch in tqdm(range(1, max_epochs + 1)):
        noise = [torch.randn(*z.shape, device=device) for z in fixed_noise]
        # update Discriminator: minimize D(G(z)) - D(x)
        for j in range(d_iter):
            # train with real
            output = discriminator(real)
            errD_real = -output.mean()
            
            # train with fake
            if j == d_iter - 1:
                fake = generate(generators, noise, reals_shapes, noise_amp)
            else:
                with torch.no_grad():
                    fake = generate(generators, noise, reals_shapes, noise_amp)
            
            output = discriminator(fake.detach())
            errD_fake = output.mean()

            gradient_penalty = calc_gradient_penalty(discriminator, real, fake.detach(), lamb, device)
            errD_total = errD_real + errD_fake + gradient_penalty
            optim_d.zero_grad()
            errD_total.backward()
            optim_d.step()

        # update Generator: minimize -D(G(z))
        output = discriminator(fake)
        errG = -output.mean()
        
        rec = generate(generators, fixed_noise, reals_shapes, noise_amp)
        rec_loss = alpha * F.mse_loss(rec, real)
        
        errG_total = errG + rec_loss
        optim_g.zero_grad()
        errG_total.backward()
        for _ in range(g_iter):
            optim_g.step()
    
        scheduler_d.step()
        scheduler_g.step()
        
        if epoch % 250 == 0:
            print(f'Epoch: {epoch: 04d}, D_real Loss: {errD_real.item():.5f}, D_fake Loss: {errD_fake.item():.5f}, '
                  f'D_gradient_penalty: {gradient_penalty.item():.5f}, G_fake Loss: {errG.item():.5f}, G_rec Loss: {rec_loss.item():.5f}')
        if epoch % 500 == 0:
            write_image(f'{output_dir}/stage_{stage:02d}/fake_sample_{epoch:04d}.jpg', fake.detach().squeeze(0))
            write_image(f'{output_dir}/stage_{stage:02d}/reconstruction_{epoch: 04d}.jpg', rec.detach().squeeze(0))
        
    torch.save(generator.state_dict(), f'{output_dir}/stage_{stage:02d}/generator.pth') 
    torch.save(discriminator.state_dict(), f'{output_dir}/stage_{stage:02d}/discriminator.pth') 

Stage: 1


 13%|█▎        | 251/2000 [00:18<02:34, 11.33it/s]

Epoch:  250, D_real Loss: -0.35454, D_fake Loss: -0.00789, D_gradient_penalty: 0.03603, G_fake Loss: 0.01947, G_rec Loss: 0.75577


 25%|██▌       | 501/2000 [00:36<01:42, 14.69it/s]

Epoch:  500, D_real Loss: -0.46850, D_fake Loss: -0.05045, D_gradient_penalty: 0.02514, G_fake Loss: 0.05241, G_rec Loss: 0.41613


...

Epoch:  1500, D_real Loss: -0.06785, D_fake Loss: 0.02166, D_gradient_penalty: 0.03237, G_fake Loss: -0.03312, G_rec Loss: 0.09761


 88%|████████▊ | 1750/2000 [22:29<03:50,  1.08it/s]

Epoch:  1750, D_real Loss: -0.07582, D_fake Loss: 0.01724, D_gradient_penalty: 0.01702, G_fake Loss: -0.01724, G_rec Loss: 0.09935


100%|██████████| 2000/2000 [25:38<00:00,  1.30it/s]

Epoch:  2000, D_real Loss: -0.07582, D_fake Loss: 0.02769, D_gradient_penalty: 0.02390, G_fake Loss: -0.02769, G_rec Loss: 0.09935


In [45]:
@torch.no_grad
def generate_samples(path: str,
                     n: int = 25) -> None:
    if not os.path.exists(path):
        os.mkdir(path)
    for idx in range(n):
        noise = [torch.randn(*z.shape, device=device) for z in fixed_noise]
        sample = generate(generators, noise, reals_shapes, noise_amp)
        write_image(f'{path}/gen_sample_{idx + 1:02d}.jpg', sample.squeeze(0))

In [46]:
generate_samples(f'{output_dir}/gen')

## Reference
1. [Official implementation](https://github.com/tamarott/SinGAN)
2. https://github.com/tohinz/ConSinGAN